In [2]:
# Autograd

import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5,3, requires_grad=True) # weights
b = torch.randn(3, requires_grad=True) # bias
z = torch.matmul(x, w) + b # y=x^T * w + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [3]:
z.grad_fn

In [4]:
loss.grad_fn

In [5]:
loss.backward()

In [8]:
w.grad # dloss/dw

tensor([[0.1658, 0.1149, 0.3229],
        [0.1658, 0.1149, 0.3229],
        [0.1658, 0.1149, 0.3229],
        [0.1658, 0.1149, 0.3229],
        [0.1658, 0.1149, 0.3229]])

In [9]:
b.grad # dloss/db

tensor([0.1658, 0.1149, 0.3229])

In [10]:
z = torch.matmul(x,w)+b

In [11]:
z.requires_grad 

True

In [16]:
# 연산 코드를 torch.no_grad() block으로 둘러싸서 연산 추적을 멈출 수 있음
with torch.no_grad():
    z = torch.matmul(x, w)+b
z.requires_grad

False

In [17]:
z = torch.matmul(x, w)+b
z_det = z.detach() # 연산 추적 멈추는 다른 방법
z_det.requires_grad

False

### gradient 추적을 멈춰야 하는 이유
### 1. 신경망의 일부 parameter를 고정된 parameter로 표시
### 2. gradient를 추적하지 않는 텐서의 연산이 더 효율적임, forward propagation만 수행할 때 연산 속도가 향상됨


### 출력 함수가 임의의 텐서인 경우, pytorch는 실제 gradient가 아닌 jacobian product를 계산함

In [21]:
inp = torch.eye(4,5, requires_grad=True)
out = (inp+1).pow(2).t()

In [22]:
out

tensor([[4., 1., 1., 1.],
        [1., 4., 1., 1.],
        [1., 1., 4., 1.],
        [1., 1., 1., 4.],
        [1., 1., 1., 1.]], grad_fn=<TBackward0>)

In [23]:
out.backward(torch.ones_like(out), retain_graph=True)
inp.grad

tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

In [24]:
out.backward(torch.ones_like(out), retain_graph=True)
inp.grad

tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

In [26]:
inp.grad.zero_()

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [27]:
out.backward(torch.ones_like(out), retain_graph=True)
inp.grad

tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

In [28]:
# 제대로된 gradient를 구하기 위해서는 grad 속성을 먼저 0으로 만들어야 함. 실제 학습에서는 optimizer가 이 과정을 도와줌